# Previsão de fakenews utilizando Naive Bayers

### Discente(s):,

Joana D'arc,  
Paulo Vitor Barbosa Santana,    
Erick Vinicius Rebouças Cruz,  
Eliezer Martins de Oliveira  


### Framework:
TensorFlow/Keras

### Requerimentos:
1. Python 3.9
2. Tensorflow (conda install -c conda-forge tensorflow-gpu=2.10)
3. Pandas
4. Matplotlib
5. Sklearn
6. Numpy
7. Dataset: https://www.kaggle.com/datasets/clmentbisaillon/fake-and-real-news-dataset  

Etapas do Projeto:
1. Carregamento e Exploração dos Dados
2. Pré-processamento
3. Otimização de Hiperparâmetros com Optuna
4. Analisando dados encontrados
5. Criação do Naive Bayers  
6. Treinamento e Validação do Modelo
7. Avaliação e Previsões  

In [14]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("clmentbisaillon/fake-and-real-news-dataset")

print("Path to dataset files:", path)


Path to dataset files: /root/.cache/kagglehub/datasets/clmentbisaillon/fake-and-real-news-dataset/versions/1


In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import ComplementNB


In [16]:
# Carregamento dos dados falsos
#path = 'data'
data_fake = pd.read_csv(path + '/Fake.csv')


print(data_fake.shape)
data_fake.head()


(23481, 4)


,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [17]:
# Carregamento dos dados verdadeiros
data_true = pd.read_csv(path + '/True.csv')
print(data_true.shape)
data_true.head()

(21417, 4)


,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [18]:
data_true['label'] = 1
data_fake['label'] = 0

In [19]:
df = pd.concat([data_true, data_fake])
df["Article"] = df["title"] + " " + df["text"]
df.sample(frac = 1) #Shuffle 100%

,title,text,subject,date,label,Article
13336,U.N. aid chief appeals for full lifting of Yem...,GENEVA (Reuters) - The United Nations appealed...,worldnews,"December 1, 2017",1,U.N. aid chief appeals for full lifting of Yem...
6414,WATCH: Donald Trump Vows To Place Anti-Aborti...,"Back in March, Donald Trump made it clear that...",News,"May 12, 2016",0,WATCH: Donald Trump Vows To Place Anti-Aborti...
1037,Cameras Aren’t Allowed In White House Press B...,Cameras have been banned from White House pres...,News,"June 23, 2017",0,Cameras Aren’t Allowed In White House Press B...
8905,Obama not involved in attorney general's decis...,WASHINGTON (Reuters) - The U.S. Justice Depart...,politicsNews,"July 1, 2016",1,Obama not involved in attorney general's decis...
6095,Trump’s new EPA transition team draws from oil...,WASHINGTON (Reuters) - U.S. President Donald T...,politicsNews,"January 24, 2017",1,Trump’s new EPA transition team draws from oil...
...,...,...,...,...,...,...
22217,Two-thirds of US Navy Strike Fighter Jets Grou...,21st Century Wire says Defense News said that ...,US_News,"February 12, 2017",0,Two-thirds of US Navy Strike Fighter Jets Grou...
16731,UK PM May says she is 'ambitious and positive'...,LONDON (Reuters) - British Prime Minister Ther...,worldnews,"October 23, 2017",1,UK PM May says she is 'ambitious and positive'...
3439,WATCH: Joy Behar And ‘The View’ PUMMEL Trump ...,The recent CIA assessment concluding that Russ...,News,"December 12, 2016",0,WATCH: Joy Behar And ‘The View’ PUMMEL Trump ...
5921,Homeland Security says monitoring litigation o...,WASHINGTON (Reuters) - The Department of Homel...,politicsNews,"January 29, 2017",1,Homeland Security says monitoring litigation o...


In [20]:
# Fazer a formatação dos textos e tokenização dos mesmos

In [21]:
import re
import nltk
# Caso ainda não tenha os stopwords baixados
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

def clean_text(text):
    # Coloca o texto em minúsculas
    text = text.lower()
    # Remove caracteres que não sejam letras e espaços
    text = re.sub(r'[^a-z\s]', '', text)
    # Remove palavras irrelevantes (stopwords)
    text = " ".join(word for word in text.split() if word not in stop_words)
    return text

# Aplica a limpeza do texto
df["Article_clean"] = df["Article"].apply(clean_text)

# Tokenização: converte os textos em uma matriz de contagem de termos
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df["Article_clean"])
y = df["label"]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 7428983 stored elements and shape (44898, 215724)>

# Divisão de dados e treinamento com Naive Bayes

In [22]:
# Divide os dados em treino e teste (80% treino, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [23]:
# Criação e treinamento do classificador Naive Bayes (utilizando ComplementNB)
clf = ComplementNB()
clf.fit(X_train, y_train)

# Realiza predições para os dados de teste
y_pred = clf.predict(X_test)

print("Matriz de Confusão:")
print(confusion_matrix(y_test, y_pred))

print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))
mcc = matthews_corrcoef(y_test, y_pred)
print("Coeficiente de Correlação de Matthews (MCC):", mcc)
clf_acc = accuracy_score(y_test, y_pred)
print("\nAcurácia:", clf_acc)


Matriz de Confusão:
[[4429  221]
 [ 149 4181]]

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.97      0.95      0.96      4650
           1       0.95      0.97      0.96      4330

    accuracy                           0.96      8980
   macro avg       0.96      0.96      0.96      8980
weighted avg       0.96      0.96      0.96      8980

Coeficiente de Correlação de Matthews (MCC): 0.9176552181494768

Acurácia: 0.9587973273942093


In [24]:
mltn = MultinomialNB()
mltn.fit(X_train, y_train)


y_pred = mltn.predict(X_test)
print("Matriz de Confusão:")
print(confusion_matrix(y_test, y_pred))

print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))
# Após obter as predições (y_pred) e os valores reais (y_test):
mcc = matthews_corrcoef(y_test, y_pred)
print("Coeficiente de Correlação de Matthews (MCC):", mcc)
mltn_acc = accuracy_score(y_test, y_pred)
print("\nAcurácia:", mltn_acc)

Matriz de Confusão:
[[4430  220]
 [ 149 4181]]

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.97      0.95      0.96      4650
           1       0.95      0.97      0.96      4330

    accuracy                           0.96      8980
   macro avg       0.96      0.96      0.96      8980
weighted avg       0.96      0.96      0.96      8980

Coeficiente de Correlação de Matthews (MCC): 0.9178742075133131

Acurácia: 0.9589086859688196


In [25]:
acc = clf_acc - mltn_acc
acc

-0.00011135857461030962

In [26]:
from sklearn.metrics import precision_score

# Para o classificador ComplementNB
precision_complement = precision_score(y_test, clf.predict(X_test))
print("Precision (ComplementNB):", precision_complement)

# Para o classificador MultinomialNB
precision_multinomial = precision_score(y_test, mltn.predict(X_test))
print("Precision (MultinomialNB):", precision_multinomial)

Precision (ComplementNB): 0.9497955474784189
Precision (MultinomialNB): 0.9500113610543058


In [27]:


# Após obter as predições (y_pred) e os valores reais (y_test):
mcc = matthews_corrcoef(y_test, y_pred)
print("Coeficiente de Correlação de Matthews (MCC):", mcc)


Coeficiente de Correlação de Matthews (MCC): 0.9178742075133131
